Import needed libraries

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import random
import os

set variables

In [ ]:
thres = 0.90 #for all points being above threshold likelihood
# there are also filepaths that can be changed at the end of the notebook

set directory path and make folder

In [ ]:
#specify starting directory as the one we are working in now with this Jupyter file
input_dir = Path.cwd()

# make folder for processed files
os.makedirs("processed_data_seperate_files", exist_ok=True)

Read in all the downsampled data files and put in big dataframe

In [ ]:
#create big dataframe
df_big = pd.DataFrame() 

# loop through video paths
for path in list(input_dir.rglob("*.csv*")):
    
    #load data
    df_load = pd.read_csv(path,header=0)
    
    # append data to big dataframe
    df_big = pd.concat([df_big, df_load], ignore_index=True)

Extract column names and get likelihood

In [ ]:
# select datacolumns that contain likelihood
headertotal = list(df_big.columns.values)
likelihood_columnindex = [index for index, string in enumerate(headertotal) if 'likelihood' in string]
likelihood_columnnames = [headertotal[x] for x in likelihood_columnindex]

Slicing the beginning of the video through 20 consistent frames above 0.90 likelihood for all bodyparts

In [ ]:
# dict for retreiving index of when data was cut for analysis purposes later
dict_cutbeginning = {}

# get index of where data is consistently above threshold for 1 second so 20 frames.
# Dataframe for begin slicing
df_begincut = pd.DataFrame()
    
# loop through df_big per video 
for vid in df_big['videoname'].unique():
    
    df_interim = df_big.loc[df_big["videoname"] == vid]
    df_interim = df_interim[likelihood_columnnames]

    def check_consecutive(row):
        return(all(row>thres))
    
    booll = df_interim.apply(check_consecutive,axis=1)  
    
    consec_count = 0
    end = None

    for i, value in enumerate(booll):
        if value:
            consec_count += 1
            if consec_count == 20:
                end = i # index of the last value of the consecutive frames
                break
        else:
            consec_count = 0
            end = None
    
    #append index per video to dict
    dict_cutbeginning[vid]= end
    
    #slicing the beginning using end_index 
    df_filter = df_big[df_big['videoname']==vid]
    df_begincut = df_begincut.append(df_filter.iloc[end + 1:])

Print smallest and largest index to show the difference of data we cut, although I checked this and it is just until the researcher put the rat in, so it is empty arena data.

In [ ]:
print('smallest index:')
print(dict_cutbeginning[min(dict_cutbeginning, key=dict_cutbeginning.get)], min(dict_cutbeginning, key=dict_cutbeginning.get))
print('largest index:')
print(dict_cutbeginning[max(dict_cutbeginning, key=dict_cutbeginning.get)], max(dict_cutbeginning, key=dict_cutbeginning.get))

Slicing of the end of the video. Looping from the end 20 frames above 0.90 likelihood, cut from the 'beginning' of the 20 frames so we don't lose data. It is just a security in case the rat's got taken out from the arena while the camera was still on. 

In [ ]:
# Dataframe for second slicing
df_secondcut = pd.DataFrame()
    
# loop through df_begincut per video 
for vid in df_begincut['videoname'].unique():
    
    df_interim2 = df_begincut.loc[df_begincut["videoname"] == vid]
    df_interim2 = df_interim2[likelihood_columnnames]
    
    # perform boolean function defined above
    booll2 = df_interim2.apply(check_consecutive,axis=1) 
    
    consec_count2 = 0
    end2 = None

    for i, value in reversed(list(enumerate(booll2))):
        if value:
            if consec_count2 == 0:
                end2 = i # index of the 'last value'
            consec_count2 += 1
            if consec_count2 == 20:
                break
        else:
            consec2_count = 0
            end2 = None
    
    #slicing the end using end_index 
    df_filter2 = df_begincut[df_begincut['videoname']==vid]
    df_secondcut = df_secondcut.append(df_filter2.iloc[:end2+1])

Select the smallest video to use as general video size

In [ ]:
# selecting smallest 'dataframe'
smallest = df_secondcut.groupby('videoname').size().min()
print(smallest)

Slice all the videos based on the smallest dataframe

In [ ]:
# slicing the end using the smallest dataframe
df_endcut = pd.DataFrame()
for vid in df_secondcut['videoname'].unique():
    df_filter3 = df_secondcut[df_secondcut['videoname']==vid]
    df_endcut = df_endcut.append(df_filter3.iloc[:smallest])

In [ ]:
# check that there is data
df_endcut.head()

# check how many videos
print(len(df_endcut['videoname'].unique()))

# check that all videos are the same length
print(df_endcut.groupby('videoname').size().mean())

Export all seperate videos after being processed

In [ ]:
# export all frames based on groupby
# file paths for exporting data
filepath = "" #edit

# export as big dataframe
df_endcut.to_csv(filepath+"sliced_bigdf.csv",index=False)

# export for single videos
for vid in df_endcut['videoname'].unique():
    filename = vid + '.csv'
    df_export_sliced = df_endcut[df_endcut['videoname']==vid]
    df_export_sliced.to_csv(filepath+filename,index=False)